In [ ]:
import delearn as dl
import torch as tt
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

dtype=tt.float32

In [ ]:
from mlp import MLP_PARAMS_DICT, MLP_PARAMS_LIST

# params = MLP_PARAMS_LIST(
#     input_size=1,
#     layer_sizes=[64,64,64],
#     output_size=1,
#     dtype=dtype,
# )

params = MLP_PARAMS_DICT(
    input_size=1,
    layer_sizes=[64,64,64],
    output_size=1,
    dtype=dtype,
)

module = dl.Module(params)

def forward(m, x): 
    return ( tt.matmul(
        tt.relu( tt.matmul(
        tt.tanh( tt.matmul(
        tt.sigmoid( tt.matmul(
        x
        , m['w0']) + m['b0'] )
        , m['w1']) + m['b1'] )
        , m['w2']) + m['b2'] )
        , m['w3']) + m['b3'] )



In [ ]:
dl.show_parameters(module)
dl.count_parameters(module)

In [ ]:
for i,(n,p) in module: print(i, n, p.shape)

In [ ]:
dl.rand_parameters(module)

In [ ]:
dl.show_parameters(module, values=True)

# Define Forward function

In [ ]:
x = tt.linspace(-5, 5, 50, dtype=dtype)
y = tt.sin(x)
ds = dl.Task( x.unsqueeze(-1), y.unsqueeze(-1), )

xx, yy = next(iter(DataLoader(ds, batch_size=len(ds), shuffle=True)))

plt.scatter(xx,yy)
print(len(ds))

In [ ]:

with tt.no_grad():
    pp = forward(module, xx)

plt.scatter(xx,yy)
plt.scatter(xx,pp)

In [ ]:
lossf = tt.nn.MSELoss()
optimf = dl.SGD(lr=0.01)

In [ ]:
import numpy as np
class TrainingCallback(dl.Callback):
    def __init__(self) -> None: super().__init__()
    def on_train_begin(self, n, optim): 
        self.losses = []
    def on_train_end(self, n): 
        plt.plot(self.losses)
    def on_batch_begin(self, batch, x, y): pass
    def on_batch_end(self, batch, batch_loss, module):self.batch_losses.append(batch_loss)
    def on_epoch_begin(self, epoch): self.batch_losses=[]
    def on_epoch_end(self, epoch): 
        self.losses.append(np.mean(self.batch_losses))
        del self.batch_losses

callback = TrainingCallback()

In [ ]:
dl.Trainer.train_ds(
    forward=forward,
    module=module,
    lossf=lossf,
    optimf=optimf,
    dsF=lambda m, n, e: ds,
    n=1000,
    batch_mode=False,
    batch_size=10,
    shuffle=True,
    callback=callback,
    create_graph=False,
)

In [ ]:
epochs = 1000
batch_size = 10
loss = tt.nn.MSELoss()

losses = []
for epoch in range(epochs):
    XY = DataLoader(ds, batch_size=batch_size, shuffle=True)
    for i,(X,Y) in enumerate(XY):
        model.module.zero_grad()
        P = model.forward(X)
        L = loss(P, Y)
        L.backward()
        losses.append(L.item())
        
        with tt.no_grad():
            for p in model.parameters(): p-=0.02*p.grad

plt.plot(losses)
